In [1]:
#python 3.6

import collections
import random
import copy
import time
import itertools
import base64

class Monoalphabetic_Solver():
    def __init__(self, ciphertext, cost_lookup):
        self.ciphertext = ciphertext
        self.cost_lookup = cost_lookup
        self.FINETUNE = 2

    def find_most_common(self):
        guesskey = [0] * 26
        text = []
        for character in self.ciphertext.lower():
            if character.isalpha():
                text.append(character)
        wordcount = collections.Counter(text)
        com = wordcount.most_common(26)
        words = ["e","t","a","o","i","n","s","h","r","d","l","c","u","m","w","f","g","y","p","b","v","k","j","x","q","z"]
        for i in range(0,26):
            guesskey[ord(com[i][0])-97] = (ord(words[i])-97)
        return guesskey

    def decode(self, key):
        plain = ""
        for x in self.ciphertext:
            if (ord(x) >= 65) & (ord(x) <= 90):
                plain += chr(65 + key[ord(x)-65])
            elif (ord(x) >= 97) & (ord(x) <= 122):
                plain += chr(97 + key[ord(x)-97])
            else:
                plain += x
        return plain

    def calculate_score(self, key):
        plaintext = self.decode(key)
        score = 0
        for i in range(len(plaintext)-2):
            try:
                score = score + self.cost_lookup[plaintext[i:i+3]]
            except:
                score = score - 50
        return score
    
    def modify_key(self, key):
        key_score = self.calculate_score(key) #calculate key score
        toswap = list(itertools.combinations(range(26), 2)) #list of ids to swap
        reset = 0 
        while len(toswap)>0: #when swap list is not empty
            a = toswap[reset][0]
            b = toswap[reset][1]
            newkey = copy.deepcopy(key)
            newkey[a] = key[b]
            newkey[b] = key[a]
            new_score = self.calculate_score(newkey) 
            if new_score>key_score: #see if after swap gets better score
                key_score = new_score
                key = newkey
                reset = 0 #if better, start with new key and repeat swap
            else:
                toswap.pop(reset) #if worse, pop the swap choise 
            reset = reset + 1
            if reset>=len(toswap):
                reset = 0

        #run through list and fine tune key
        for n in range(0,self.FINETUNE):
            toswap = list(itertools.combinations(range(26), 2))
            for reset in range(0,len(toswap)):
                a = toswap[reset][0]
                b = toswap[reset][1]
                newkey = copy.deepcopy(key)
                newkey[a] = key[b]
                newkey[b] = key[a]
                new_score = self.calculate_score(newkey)
                if new_score>key_score:
                    key_score = new_score
                    key = newkey

        return key, key_score

    def find_key(self):
        guesskey = self.find_most_common()
        key, score = self.modify_key(guesskey)
        return key


class Monoalphabetic_Solver_Base64():
    def __init__(self, ciphertext, cost_lookup):
        self.ciphertext = ciphertext
        self.cost_lookup = cost_lookup
        self.FINETUNE = 2

    def isBase64(self, text):
        try:
            temp_text = text.encode()
            return base64.b64encode(base64.b64decode(temp_text)) == text.encode()
        except Exception:
            return False

    def decode(self, key):
        plain = ""
        for x in self.ciphertext:
            if (ord(x) >= 65) & (ord(x) <= 90):
                plain += chr(65 + key[ord(x)-65])
            elif (ord(x) >= 97) & (ord(x) <= 122):
                plain += chr(97 + key[ord(x)-97])
            else:
                plain += x
        return plain
    
    def calculate_score(self, key):
        score = 0
        plain = self.decode(key)
        for i in range(int(len(plain)/4)):
            if self.isBase64(plain[i*4:i*4+4]):
                try:
                    temp = plain[i*4:i*4+4].encode()
                    temp = base64.b64decode(temp)
                    temp = temp.decode()
                    score = score + self.cost_lookup[temp]
                except:
                    score = score - 50
            else:
                score = score - 50
        return score

    def modify_key(self, key):
        key_score = self.calculate_score(key)
        toswap = list(itertools.combinations(range(26), 2))
        reset = 0
        while len(toswap)>0:
            a = toswap[reset][0]
            b = toswap[reset][1]
            newkey = copy.deepcopy(key)
            newkey[a] = key[b]
            newkey[b] = key[a]
            new_score = self.calculate_score(newkey)
            if new_score>key_score:
                key_score = new_score
                key = newkey
                reset = 0
            else:
                toswap.pop(reset)
            reset = reset + 1
            if reset>=len(toswap):
                reset = 0

        #fine tune key
        for n in range(0,self.FINETUNE):
            toswap = list(itertools.combinations(range(26), 2))
            for reset in range(0,len(toswap)):
                a = toswap[reset][0]
                b = toswap[reset][1]
                newkey = copy.deepcopy(key)
                newkey[a] = key[b]
                newkey[b] = key[a]
                new_score = self.calculate_score(newkey)
                if new_score>key_score:
                    key_score = new_score
                    key = newkey

        return key, key_score

    def find_key(self):
        guesskey = []
        for i in range(0,26):
            guesskey.append(i)
        random.shuffle(guesskey)
        key, score = self.modify_key(guesskey)
        return key
    
def Solve_Monoalphabetic(textfile):
    with open(textfile) as f:
        ciphertext = f.read()

    cost_lookup = {}
    with open("three_no_spaces_freqs.txt") as f:
        for line in f:
            (key, val) = line.split()
            cost_lookup[key] = float(val)

    mono = Monoalphabetic_Solver(ciphertext,cost_lookup)
    start = time.time()
    key = mono.find_key()
    end = time.time()
    print(mono.decode(key))

    ch = [0]*26
    for i in key:
        ch[key[i]] = chr(i+97).upper()

    print("Key : ","".join(ch)," Time : ",end-start)


def Solve_Monoalphabetic_Base64(textfile):
    with open(textfile) as f:
        ciphertext = f.read()
    
    cost_lookup = {}
    with open("three_no_spaces_freqs.txt") as f:
        for line in f:
            (key, val) = line.split()
            cost_lookup[key] = float(val)
    
    mono = Monoalphabetic_Solver_Base64(ciphertext,cost_lookup)
    start = time.time()
    key = mono.find_key()
    end = time.time()
    text = mono.decode(key)
    print(base64.b64decode(text))

    ch = [0]*26
    for i in key:
        ch[key[i]] = chr(i+97).upper()

    print("Key : ","".join(ch)," Time : ",end-start)

if __name__== "__main__":
    Solve_Monoalphabetic("textfile/ciphertext1.txt")
    #Solve_Monoalphabetic_Base64("Base64.txt")

{BKPWTHPSOBXZTRUVEIW}burneyhersistersletterbeforereayingitSerRoyriktuggeyathiswhiskersPoisonwellthatcoulybetheywarfsworktrueenoughOrCerseisItssaiypoisonisawomansweaponbeggingdourparyonsmdlaydTheKingsladernowIhavenogreatlikingforthemanbuthesnotthesortToofonyofthesightofblooyonthatgolyensworyofhisWasitpoisonmdlaydCateldnfrowneyvaguelduneasdHowelsecoulythedmakeitlookanaturalyeathBehinyherLoryRobertshriekeywithyelightasoneofthepuppetknightssliceytheotherinhalfspillingaflooyofreysawyustontotheterraceSheglanceyathernephewanysigheyThebodisutterldwithoutyisciplineHewillneverbestrongenoughtoruleunlessheistakenawadfromhismotherforatimeHisloryfatheragreeywithdousaiyavoiceatherelbowSheturneytobeholyMaesterColemonacupofwineinhishanyHewasplanningtosenythebodtoYragonstoneforfosteringdouknowohbutImspeakingoutofturnTheappleofhisthroatbobbeyanxiousldbeneaththeloosemaesterschainIfearIvehaytoomuchofLoryHuntersexcellentwineTheprospectofblooysheyhasmdnervesallafradDouaremistakenMaesterCateldnsaiyItwasCaster